In [1]:
! pip install rdkit pysmiles prettytable pybel huggingface_hub ijson datasets

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from prettytable import PrettyTable
import multiprocessing
from scipy import stats
import os, sys
import time
import json

In [3]:
start = time.time()

file_dir = os.path.dirname("../")
sys.path.append(file_dir)

from anima.smiles import SMILES

sml = SMILES()

cores = int(multiprocessing.cpu_count())
print(f"Cores: {cores}, GPUs: {torch.cuda.device_count()}")

# Checking CUDA
torch.cuda.empty_cache()
use_cuda = True
print(f"Cuda available: {torch.cuda.is_available()}")
device = torch.device(
    "cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

Cores: 24, GPUs: 1
Cuda available: True


In [4]:
database = pd.read_csv("../anima-master/databases/OMEAD_41801.csv")

print(f"\nDatabase shape: {database.shape}\n")

train_db = database
train_db["homo_lumo_gap"] = train_db.lumo - train_db.homo
smiles = np.array(train_db.smiles)
#train_db.to_csv("changed_test200k.csv")
print(f"\nNew size of train database: {train_db.shape}\n")


# Define SMILES vocabulary

print("\nDefining SMILES vocabulary\n")
vocab = sml.smilesVOC(smiles, n_jobs=cores)

vocab_size = len(vocab)

print(f"\nSize of vocabulary: {vocab_size}\n")

# Saving the vocabulary
with open('vocab.dat', 'w') as f:
    for i in vocab:
        f.write(str(i) + '\n')
    f.close()


Database shape: (41801, 20)


New size of train database: (41801, 21)


Defining SMILES vocabulary



[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    1.1s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    1.4s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    1.8s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    2.7s
[Parallel(n_jobs=24)]: Done 2402 tasks      | elapsed:    3.8s
[Parallel(n_jobs=24)]: Done 3152 tasks      | elapsed:    6.3s
[Parallel(n_jobs=24)]: Done 4002 tasks      | elapsed:    9.1s
[Parallel(n_jobs=24)]: Done 4952 tasks      | elapsed:   12.2s
[Parallel(n_jobs=24)]: Done 6002 tasks      | elapsed:   15.8s
[Parallel(n_jobs=24)]: Done 7152 tasks      | elapsed:   19.8s
[Parallel(n_jobs=24)]: Done 8402 tasks      | elapsed:   24.0s
[Parallel(n_jobs=24)]: Done 9752 tasks      | elapsed:   28.7s
[Parallel(n_jobs=24)]: Done 11202 tasks 


Size of vocabulary: 33



In [5]:
# Preparing SMILES into SEQUENCES or 1h tensors
# for embeddings
all_sequences = []

l = 1
for i in smiles:
    print(l, end="\r")
    all_sequences.append(torch.tensor(sml.smilesToSequence(i, vocab)))
    l += 1

# for embeddings
packing = torch.nn.utils.rnn.pack_sequence(
    all_sequences,
    enforce_sorted = False
)

packing_padding = torch.nn.utils.rnn.pad_packed_sequence(
    packing,
    batch_first = True
)


# Check dimensions
# for embedding
# BATCH x SEQUENCE x INFO
print(f"\nCheck packing shape: {packing_padding[0][:,:,:].size()}\n")


temp = packing_padding[0][:,:,0]
random_state = 1
test_size = 0.12

# inputs
x_train, x_test = train_test_split(
    temp.numpy(),
    test_size=test_size,
    random_state=1
)
del temp

gap = np.array(train_db.homo_lumo_gap)
# targets / outputs
y_train, y_test = train_test_split(
    gap,
    test_size=test_size,
    random_state=1
)

max_length = x_train.shape[-1]
# Store max length for prediction model
config = {"max_length": max_length}

with open('config.json', 'w') as f:
    json.dump(config, f)
    f.close()


# max length of tensor sequences
print(f"\nMax length of tensor sequences: {x_train.shape[-1]}")

print(f"\nTest size: {x_test.shape}\n")
print(f"\nTrain size: {x_train.shape}\n")

41801
Check packing shape: torch.Size([41801, 224, 1])


Max length of tensor sequences: 224

Test size: (5017, 224)


Train size: (36784, 224)



In [6]:
# Define TOOLS
def model_evaluation(model, x_test, y_test, device):
    batch_size=64

    test_data = TensorDataset(
        torch.tensor(x_test),
        torch.tensor(y_test)
    )

    test_loader = DataLoader(
        test_data,
        shuffle= False,
        batch_size= batch_size,
        drop_last = False
    )

    running_mae = []
    running_mse = []
    mae = torch.nn.L1Loss().to(device)
    mse = torch.nn.SmoothL1Loss().to(device)

    model.eval().to(device)

    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.double()

            output = model(inputs)

            loss_mae = mae(output, targets)
            loss_mse = mse(output, targets)

            running_mae.append(loss_mae.item())
            running_mse.append(loss_mse.item())

    model.train()
    return np.mean(running_mae), np.mean(running_mse)


def model_predictions(model, x, device, batch_size = 64):
    # predictions

    model.eval().to(device)

    pred_data = TensorDataset(torch.tensor(x))
    pred_loader = DataLoader(
        pred_data,
        shuffle=False,
        batch_size=batch_size,
        drop_last=False,
    )

    batches = len(x) / batch_size

    with torch.no_grad():
        for batch_idx, data in enumerate(pred_loader):
            print(f"Batch: {batch_idx + 1:010.2f} of {batches:010.2f}", end="\r")
            inputs = data[0]
            inputs = inputs.to(device)

            output = model(inputs)
            if batch_idx == 0:
                temp = output.cpu().detach().numpy()
            else:
                temp = np.append(temp, output.cpu().detach().numpy())
            del inputs, output

    return np.reshape(temp, -1)

In [7]:
class SMILESTransformerRegressor(nn.Module):
    def __init__(self, vocab_size, embed_dim, n_heads, n_layers, hidden_dim, max_len, output_dim, dropout):
        super(SMILESTransformerRegressor, self).__init__()

        self.embeddings = nn.Embedding(vocab_size + 1, embed_dim, padding_idx=0)

        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.randn(1, max_len, embed_dim))

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Transformer encoding
        encoder_layer = nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=n_heads,
                dim_feedforward=hidden_dim,
                batch_first=True
                )

        self.transformer_encoder = nn.TransformerEncoder(
                encoder_layer,
                num_layers=n_layers)

        # Regression head
        self.fc = nn.Sequential(
                nn.Linear(embed_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, output_dim)
                )

    def forward(self, x):
         """
         x: Tensor of shape (batch_size, seq_len)
         """
         batch_size, seq_len = x.shape

         # Embed and add positional encoding
         x_embed = self.embeddings(x) # (batch size, seq_len, embed_dim)

         # If sequence is shorter than max_len, crop pos encoding
         pos_enc = self.positional_encoding[:, :seq_len, :].to(x.device) # (1, seq_len, embed_dim)
         x_embed = x_embed + pos_enc

         # Transformer expects input shape (batch_size, seq_len, embed_dim)
         x_transformed = self.transformer_encoder(x_embed)

         # Pooling: mean over sequence dimension
         x_pooled = x_transformed.mean(dim=1) # (batch_size, embed_dim)

         out = self.fc(x_pooled) # (batch_size, 1)
         return out.squeeze(1) # (batch_size, )

In [ ]:
epochs = 2000
all_data = len(x_train)
log = 10

# Read best params from evo algorithm
with open('best_param.json', 'r') as f:
    best_params = json.load(f)
    f.close()

embed_dim = best_params["embed_dim"]
n_heads = best_params["n_heads"]
n_layers = best_params["n_layers"]
hidden_dim = best_params["hidden_dim"]
output_dim = 1
weight_decay = best_params["weight_decay"]
batch_size = best_params["batch_size"]
dropout = best_params["dropout"]

#embed_dim = 128
#n_heads = 4
#n_layers = 2
#hidden_dim = 256
#output_dim = 1
#weight_decay = 1e-5
#batch_size = 64

nn_gap = SMILESTransformerRegressor(vocab_size, embed_dim, n_heads, n_layers, hidden_dim, max_length, output_dim, dropout)
nn_gap.float().to(device)

# loss
metric = torch.nn.SmoothL1Loss().double()
optimizer = torch.optim.Adam(nn_gap.parameters(), lr=0.001, weight_decay=weight_decay)

# setting the dataloader
train_data = TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=False, pin_memory=True)


epoch_loss = []
test_loss = []

for epoch in range(epochs):
    nn_gap.train().to(device)
    
    running_loss = []
    running_metric = []
    for batch_idx, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0]
        targets = data[1]
        inputs, targets = inputs.to(device), targets.to(device)
        targets = targets.double()

        optimizer.zero_grad()

        output = nn_gap(inputs)

        loss = torch.mean(torch.abs(targets - output))
        loss.backward()
        optimizer.step()

        # Print statistics every log
        running_loss.append(loss.item())
        running_metric.append(metric(output, targets).item())
        if batch_idx % log == 0:
            print(f"Epoch {epoch+1:04d} [{batch_idx * len(inputs):05d}/{all_data:05d} ({100 * batch_idx * batch_size / len(train_data):04.1f})%] \t\t Loss: {np.mean(running_loss):02.3f}", end="\r")

    epoch_loss.append(np.mean(running_loss))
    test_mae, test_mse = model_evaluation(nn_gap, x_test, y_test, device)
    test_loss.append(test_mae)

    print(f"Epoch: {epoch + 1:04d} [] Train (Loss={np.mean(running_loss):02.3f} | Metric={np.mean(running_metric):02.3f})\tVal. (MAE={test_mae:02.3f} | HL={test_mse:02.3f})", end="\n")

print("\n")
print(f"\n Train Loss: {round(epoch_loss[-1], 2)}")
print(f"\n Test MAE: {round(test_loss[-1], 2)}")
print("\n Finished training")

Epoch: 0001 [] Train (Loss=0.033 | Metric=0.003)	Val. (MAE=0.018 | HL=0.000)
Epoch: 0002 [] Train (Loss=0.018 | Metric=0.000)	Val. (MAE=0.023 | HL=0.000)
Epoch: 0003 [] Train (Loss=0.018 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0004 [] Train (Loss=0.017 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0005 [] Train (Loss=0.017 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0006 [] Train (Loss=0.017 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0007 [] Train (Loss=0.017 | Metric=0.000)	Val. (MAE=0.018 | HL=0.000)
Epoch: 0008 [] Train (Loss=0.017 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0009 [] Train (Loss=0.017 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0010 [] Train (Loss=0.016 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0011 [] Train (Loss=0.016 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0012 [] Train (Loss=0.016 | Metric=0.000)	Val. (MAE=0.019 | HL=0.000)
Epoch: 0013 [] Train (Loss=0.016 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch: 0222 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0223 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0224 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0225 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0226 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0227 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0228 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0229 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0230 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0231 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0232 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)
Epoch: 0233 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0234 [] Train (Loss=0.013 | Metric=0.000)	Val. (MAE=0.015 | HL=0.000)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch: 0954 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0955 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0956 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0957 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0958 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0959 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0960 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0961 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0962 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0963 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0964 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)
Epoch: 0965 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.016 | HL=0.000)
Epoch: 0966 [] Train (Loss=0.009 | Metric=0.000)	Val. (MAE=0.017 | HL=0.000)

In [ ]:
# Plot training and test errors
plt.plot(range(epoch + 1), test_loss, 'o', c='black', ms=5, label='Test')
plt.plot(range(epoch + 1), epoch_loss, 'o', c='red', ms=5, label='Training')
plt.xlabel("n of epochs", size=14)
plt.ylabel("MAE", size=14)
plt.legend(fontsize=14)
plt.tight_layout()
plt.savefig("Train_test_errors.png", dpi=200)


predictions = model_predictions(nn_gap, x_test, device, batch_size=64)

px = predictions.reshape(-1)
py = y_test.reshape(-1)

# Generate linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(px, py)
line = slope * px + intercept
conf = std_err * 2.58  # 99% confidence interval

fig, axs = plt.subplots(1, 1, dpi=80)
axs.tick_params(axis="both", which="major", labelsize=12)

axs.plot(px, py, "o", color="xkcd:tomato", aa=True, alpha=0.6)
axs.plot(px, line, color="xkcd:azure", aa=True, alpha=0.6)

axs.set_ylabel(r"Target Values (Hartree)", fontsize=14)
axs.set_xlabel("Predicted Values (Hartree)", fontsize=14)
fig.text(0.13, 0.90, "R$^2$ = " + str(round(r_value**2, 2)), ha="left", fontsize=13)

fig.tight_layout()
plt.savefig("Linear_reg.png", dpi=200)

# Save the model
torch.save('nn_gap.pt')